# Data Wrangling #

## Introduction ##

This project focused on wrangling data from the WeRateDogs Twitter account using Python, documented in a Jupyter Notebook (wrangle_act.ipynb). This Twitter account rates dogs with humorous commentary. The rating denominator is usually 10, however, the numerators are usually greater than 10. They’re Good Dogs Brent wrangle WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. WeRateDogs has over 8 million followers and has received international media coverage.

The goal of this project is to wrangle the WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. The challenge lies in the fact that the Twitter archive is great, but it only contains very basic tweet information that comes in JSON format. I needed to gather, asses and clean the Twitter data for a worthy analysis and visualization.

## Project Details ##

Fully assessing and cleaning the entire dataset would require exceptional effort so only a subset of its issues (eight quality issues and two tidiness issues at minimum) needed to be assessed and cleaned.

The tasks for this project were:

Data wrangling, which consists of:
- Gathering data
- Assessing data
- Cleaning data
- Storing, analyzing, and visualizing our wrangled data
- Reporting on 1) our data wrangling efforts and 2) our data analyses and visualizations


## Gathering Data ##

1)Twitter archive file: download this file manually by clicking the following link: twitter_archive_enhanced.csv

2)The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (image_predictions.tsv) is hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

3)Twitter API & JSON: Each tweet's retweet count and favorite ("like") count at minimum, and any additional data you find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count.



<b>1.Twitter archive file<b>

In [1]:
# Importing all the required packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import json
import requests 
import re
import tweepy
import time
import warnings

In [2]:
#Reading the CSV file
twitter_archive_df = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
#Checking out the contents of the dataframe
twitter_archive_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
#Checking the structure of the dataframe
twitter_archive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

<b>2. Tweet image prediction

In [5]:
#Use requests library to download tsv file
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

response = requests.get(url)

with open(os.path.join('image_predictions.tsv'), mode = 'wb') as file:
    file.write(response.content)

In [6]:
#Reading the TSV file
image_prediction_df = pd.read_csv('image_predictions.tsv', sep = '\t')

In [7]:
#Checking the structure of the images dataframe
image_prediction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


<b>3.Query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file.

In [8]:
#Keys needed for authorisation of API
consumer_key = 'jKIgiqqCO0Z8vetxw6FTDcRBL'
consumer_secret = 'cOa6MPsGoGd0QlvgRegqwTs92P62SQDKEVYTuJ4Tj7lrJDJF52'
access_token = '2910967932-Y4aELTTmPEORAsnhMjuW4MZPEBARod8cpjLKQkf'
access_secret = 'azcXAYJz2hslCemvJTclZxiwDDExTqaoEOlb2R6eD9K4I'

In [9]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

In [10]:
api = tweepy.API(auth, 
                 parser = tweepy.parsers.JSONParser(), 
                 wait_on_rate_limit = True, 
                 wait_on_rate_limit_notify = True)

In [11]:
#Download Tweepy status object based on Tweet ID and store in list
list_of_tweets = []
# Tweets that can't be found are saved in the list below:
cant_find_tweets_for_those_ids = []
for tweet_id in twitter_archive_df['tweet_id']:   
    try:
        list_of_tweets.append(api.get_status(tweet_id))
    except Exception as e:
        cant_find_tweets_for_those_ids.append(tweet_id)

Rate limit reached. Sleeping for: 289
Rate limit reached. Sleeping for: 387


In [12]:
print("The list of tweets" ,len(list_of_tweets))
print("The list of tweets not found" , len(cant_find_tweets_for_those_ids))

The list of tweets 2333
The list of tweets not found 23


In [13]:
#Then in this code block we isolate the json part of each tweepy 
#status object that we have downloaded and we add them all into a list

my_list_of_dicts = []
for each_json_tweet in list_of_tweets:
    my_list_of_dicts.append(each_json_tweet)

In [14]:
#we write this list into a txt file:

with open('tweet_json.txt', 'w') as file:
        file.write(json.dumps(my_list_of_dicts, indent=4))

In [15]:
#identify information of interest from JSON dictionaries in txt file
#and put it in a dataframe called tweet JSON
my_demo_list = []
with open('tweet_json.txt', encoding='utf-8') as json_file:  
    all_data = json.load(json_file)
    for each_dictionary in all_data:
        tweet_id = each_dictionary['id']
        whole_tweet = each_dictionary['text']
        only_url = whole_tweet[whole_tweet.find('https'):]
        favorite_count = each_dictionary['favorite_count']
        retweet_count = each_dictionary['retweet_count']
        followers_count = each_dictionary['user']['followers_count']
        friends_count = each_dictionary['user']['friends_count']
        whole_source = each_dictionary['source']
        only_device = whole_source[whole_source.find('rel="nofollow">') + 15:-4]
        source = only_device
        retweeted_status = each_dictionary['retweeted_status'] = each_dictionary.get('retweeted_status', 'Original tweet')
        if retweeted_status == 'Original tweet':
            url = only_url
        else:
            retweeted_status = 'This is a retweet'
            url = 'This is a retweet'

        my_demo_list.append({'tweet_id': str(tweet_id),
                             'favorite_count': int(favorite_count),
                             'retweet_count': int(retweet_count),
                             'followers_count': int(followers_count),
                             'friends_count': int(friends_count),
                             'url': url,
                             'source': source,
                             'retweeted_status': retweeted_status,
                            })
        tweet_json = pd.DataFrame(my_demo_list, columns = ['tweet_id', 'favorite_count','retweet_count', 
                                                           'followers_count', 'friends_count','source', 
                                                           'retweeted_status', 'url'])

In [16]:
tweet_json.head()

,tweet_id,favorite_count,retweet_count,followers_count,friends_count,source,retweeted_status,url
0,892420643555336193,37375,8074,8188694,12,Twitter for iPhone,Original tweet,https://t.co/MgUWQ76dJU
1,892177421306343426,32135,5993,8188694,12,Twitter for iPhone,Original tweet,https://t.co/aQFSeaCu9L
2,891815181378084864,24221,3963,8188694,12,Twitter for iPhone,Original tweet,https://t.co/r0YlrsGCgy
3,891689557279858688,40685,8234,8188695,12,Twitter for iPhone,Original tweet,https://t.co/tD36da7qLQ
4,891327558926688256,38901,8919,8188695,12,Twitter for iPhone,Original tweet,https://t.co/0g0KMIVXZ3


In [17]:
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2333 entries, 0 to 2332
Data columns (total 8 columns):
tweet_id            2333 non-null object
favorite_count      2333 non-null int64
retweet_count       2333 non-null int64
followers_count     2333 non-null int64
friends_count       2333 non-null int64
source              2333 non-null object
retweeted_status    2333 non-null object
url                 2333 non-null object
dtypes: int64(4), object(4)
memory usage: 145.9+ KB


### Gathering Summary:

Gathering is the first step in the data wrangling process.

Obtaining data
- Getting data from an existing file (twitter-archive-enhanced.csv) Reading from csv file using pandas
- Downloading a file from the internet (image-predictions.tsv) Downloading file using requests
- Querying an API (tweet_json.txt) Get JSON object of all the tweet_ids using Tweepy
- Importing that data into our programming environment (Jupyter Notebook)


## Assessing Data

### Visual Assessment 

<b>Each dataframe obtained is displayed in this Jupyter Notebook for visual assessment purposes.

In [18]:
#Viewing some random samples from the dataframes
twitter_archive_df.sample(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
325,833863086058651648,NaN,NaN,2017-02-21 02:17:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bentley. Hairbrushes are his favorite ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/833863086...,12,10,Bentley,None,None,None,None
354,831262627380748289,NaN,NaN,2017-02-13 22:03:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tucker. He's feeling h*ckin festive an...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/831262627...,12,10,Tucker,None,None,None,None
1093,737310737551491075,NaN,NaN,2016-05-30 15:52:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Everybody stop what you're doing and watch the...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/737310737...,13,10,None,None,None,None,None
1880,675006312288268288,NaN,NaN,2015-12-10 17:37:00 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Mollie. This pic was taken after ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675006312...,10,10,Mollie,None,None,pupper,None
408,823581115634085888,NaN,NaN,2017-01-23 17:20:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Crawford. He's quite h*ckin good at th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/823581115...,11,10,Crawford,None,None,None,None
1583,687102708889812993,NaN,NaN,2016-01-13 02:43:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Army of water dogs here. None of them know whe...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/687102708...,5,10,None,None,None,None,None
2292,667160273090932737,NaN,NaN,2015-11-19 01:59:39 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bradlay. He is a Ronaldinho Matsuyama ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667160273...,11,10,Bradlay,None,None,None,None
846,766313316352462849,NaN,NaN,2016-08-18 16:38:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Oscar. He has legendary eyebrows and h...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/766313316...,12,10,Oscar,None,None,None,None
1900,674664755118911488,NaN,NaN,2015-12-09 18:59:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rodman. He's getting destroyed by the ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/674664755...,10,10,Rodman,None,None,None,None
1031,745422732645535745,NaN,NaN,2016-06-22 01:06:43 +0000,"<a href=""http://twitter.com/download/iphone"" r...",We only rate dogs. Pls stop sending in non-can...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/745422732...,9,10,very,None,None,None,None


In [19]:
image_prediction_df.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
477,675166823650848770,https://pbs.twimg.com/media/CV6spB7XAAIpMyP.jpg,1,llama,0.284394,False,standard_poodle,0.132569,True,teddy,0.127975,False
1755,824796380199809024,https://pbs.twimg.com/media/CwiuEJmW8AAZnit.jpg,2,gas_pump,0.676439,False,harvester,0.049995,False,swing,0.044660,False
1462,778383385161035776,https://pbs.twimg.com/media/Cs1fjyqWIAE2jop.jpg,1,collie,0.345266,True,borzoi,0.312823,True,Border_collie,0.213011,True
1142,729854734790754305,https://pbs.twimg.com/media/CiD3AfkXEAA3S_r.jpg,1,doormat,0.359586,False,china_cabinet,0.053901,False,passenger_car,0.052665,False
578,678755239630127104,https://pbs.twimg.com/media/CWtsSQAUkAAnWws.jpg,1,malamute,0.606654,True,Border_collie,0.193831,True,collie,0.048378,True
1382,765395769549590528,https://pbs.twimg.com/media/Cp87Y0jXYAQyjuV.jpg,1,Pembroke,0.509491,True,Cardigan,0.330401,True,Shetland_sheepdog,0.038875,True
870,697881462549430272,https://pbs.twimg.com/media/Ca9feqDUAAA_z7T.jpg,1,washbasin,0.176423,False,paper_towel,0.167462,False,toilet_tissue,0.098029,False
1302,752917284578922496,https://pbs.twimg.com/media/CnLmRiYXEAAO_8f.jpg,1,German_shepherd,0.609283,True,malinois,0.352460,True,kelpie,0.016105,True
1313,754747087846248448,https://pbs.twimg.com/media/CnlmeL3WgAA4c84.jpg,1,rotisserie,0.471493,False,cash_machine,0.250837,False,sliding_door,0.117872,False
847,695409464418041856,https://pbs.twimg.com/media/CaaXN5LUYAEzAh-.jpg,1,pug,0.997445,True,bull_mastiff,0.001749,True,Pekinese,0.000304,True


In [20]:
tweet_json.sample(10)

,tweet_id,favorite_count,retweet_count,followers_count,friends_count,source,retweeted_status,url
1310,705475953783398401,3032,953,8188724,12,Twitter for iPhone,Original tweet,https://t.co/zQUuVu812n
915,753298634498793472,0,6034,8188719,12,Twitter for iPhone,This is a retweet,This is a retweet
1387,699060279947165696,3877,1875,8188725,12,Vine - Make a Scene,Original tweet,https://t.co/n6wND1v7il
1893,674307341513269249,11357,7046,8188733,12,Vine - Make a Scene,Original tweet,https://t.co/SroTpI6psB
949,750101899009982464,3149,884,8188719,12,Twitter for iPhone,Original tweet,https://t.co/enCpXzZHkD
1187,715696743237730304,4008,1336,8188722,12,Twitter for iPhone,Original tweet,https://t.co/cYPHuJYTjC
507,809084759137812480,13903,3761,8188711,12,Twitter for iPhone,Original tweet,https://t.co/ySn6zB6mwQ
1679,680940246314430465,3298,1095,8188727,12,Twitter for iPhone,Original tweet,https://t.co/VQXdwJfkyS
116,869227993411051520,19965,3673,8188698,12,Twitter for iPhone,Original tweet,https://t.co/2lERR4Uoyt
1255,709042156699303936,4887,1699,8188723,12,Twitter for iPhone,Original tweet,https://t.co/7BkkVNu5pd


### Programmatic Assessment

<b>Now, that we have assessed the data visually. It's time to assess the data programmatically using Python's libraries.

In [21]:
twitter_archive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [22]:
twitter_archive_df.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [23]:
twitter_archive_df['name'].value_counts()

None          745
a              55
Charlie        12
Oliver         11
Cooper         11
Lucy           11
Penny          10
Tucker         10
Lola           10
Bo              9
Winston         9
Sadie           8
the             8
Daisy           7
an              7
Toby            7
Buddy           7
Bailey          7
Milo            6
Rusty           6
Bella           6
Dave            6
Leo             6
Scout           6
Jack            6
Oscar           6
Stanley         6
Jax             6
Koda            6
George          5
             ... 
Caryl           1
Tess            1
Creg            1
Jazzy           1
Major           1
Mojo            1
Brandonald      1
Liam            1
Marq            1
Shelby          1
Sundance        1
Napolean        1
Grady           1
Tayzie          1
Jeffri          1
Ester           1
DayZ            1
Brandy          1
Strudel         1
Coleman         1
Jonah           1
Stormy          1
Bobby           1
Dewey           1
Chuq      

In [24]:
twitter_archive_df['rating_numerator'].value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [25]:
#This will print the location of the tweet i.e tweet_id along with the text
print(twitter_archive_df.loc[twitter_archive_df.rating_numerator == 143, 'text'])
print(twitter_archive_df.loc[twitter_archive_df.rating_numerator == 204, 'text'])
print(twitter_archive_df.loc[twitter_archive_df.rating_numerator == 1176, 'text'])
print(twitter_archive_df.loc[twitter_archive_df.rating_numerator == 144, 'text'])
print(twitter_archive_df.loc[twitter_archive_df.rating_numerator == 666, 'text'])

1634    Two sneaky puppers were not initially seen, mo...
Name: text, dtype: object
1120    Say hello to this unbelievably well behaved sq...
Name: text, dtype: object
Series([], Name: text, dtype: object)
1779    IT'S PUPPERGEDDON. Total of 144/120 ...I think...
Name: text, dtype: object
189    @s8n You tried very hard to portray this good ...
Name: text, dtype: object


In [26]:
#We can now verify if the tweet's text contains that numerator by entering the tweet_id
print(twitter_archive_df['text'][1634])
print(twitter_archive_df['text'][1120])
print(twitter_archive_df['text'][1779])
print(twitter_archive_df['text'][189])

Two sneaky puppers were not initially seen, moving the rating to 143/130. Please forgive us. Thank you https://t.co/kRK51Y5ac3
Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv
IT'S PUPPERGEDDON. Total of 144/120 ...I think https://t.co/ZanVtAtvIq
@s8n You tried very hard to portray this good boy as not so good, but you have ultimately failed. His goodness shines through. 666/10


In [27]:
twitter_archive_df['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [28]:
print(twitter_archive_df.loc[twitter_archive_df.rating_denominator == 11, 'text'])
print(twitter_archive_df.loc[twitter_archive_df.rating_denominator == 16, 'text'])
print(twitter_archive_df.loc[twitter_archive_df.rating_denominator == 15, 'text'])
print(twitter_archive_df.loc[twitter_archive_df.rating_denominator == 7, 'text'])
print(twitter_archive_df.loc[twitter_archive_df.rating_denominator == 2, 'text'])

784     RT @dog_rates: After so many requests, this is...
1068    After so many requests, this is Bretagne. She ...
1662    This is Darrel. He just robbed a 7/11 and is i...
Name: text, dtype: object
1663    I'm aware that I could've said 20/16, but here...
Name: text, dtype: object
342    @docmisterio account started on 11/15/15
Name: text, dtype: object
516    Meet Sam. She smiles 24/7 &amp; secretly aspir...
Name: text, dtype: object
2335    This is an Albanian 3 1/2 legged  Episcopalian...
Name: text, dtype: object


In [29]:
print(twitter_archive_df['text'][784])  #It's clearly a retweet
print(twitter_archive_df['text'][1068]) #9/11 is the date. Actual rating is 14/10
print(twitter_archive_df['text'][1662]) #7/11 is the store. Actual rating is 10/10
print(twitter_archive_df['text'][1663]) # It's a perfect tweet. No changes need to be made.
print(twitter_archive_df['text'][342]) # No rating is mentioned here
print(twitter_archive_df['text'][516]) # No rating is mentioned here
print(twitter_archive_df['text'][2335]) #Here, the rating is 9/10

RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…
After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ
This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5
I'm aware that I could've said 20/16, but here at WeRateDogs we are very professional. An inconsistent rating scale is simply irresponsible
@docmisterio account started on 11/15/15
Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. 
Keep Sam smiling by clicking and sharing this link:
https://t.co/98tB8y7y7t https://t.co/LouL5vdvxx
This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv


In [30]:
# Source of each tweet
twitter_archive_df['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [31]:
# Rating having numerator above 20
twitter_archive_df[twitter_archive_df['rating_numerator'] > 20]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None
189,855860136149123072,8.558585e+17,1.361572e+07,2017-04-22 19:05:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@s8n You tried very hard to portray this good ...,NaN,NaN,NaN,NaN,666,10,None,None,None,None,None
290,838150277551247360,8.381455e+17,2.195506e+07,2017-03-04 22:12:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@markhoppus 182/10,NaN,NaN,NaN,NaN,182,10,None,None,None,None,None
313,835246439529840640,8.352460e+17,2.625958e+07,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
340,832215909146226688,NaN,NaN,2017-02-16 13:11:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: This is Logan, the Chow who liv...",7.867091e+17,4.196984e+09,2016-10-13 23:23:56 +0000,https://twitter.com/dog_rates/status/786709082...,75,10,Logan,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
695,786709082849828864,NaN,NaN,2016-10-13 23:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Logan, the Chow who lived. He solemnly...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/786709082...,75,10,Logan,None,None,None,None
763,778027034220126208,NaN,NaN,2016-09-20 00:24:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sophie. She's a Jubilant Bush Pupper. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/778027034...,27,10,Sophie,None,None,pupper,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244...,165,150,None,None,None,None,None


In [32]:
twitter_archive_df[twitter_archive_df['retweeted_status_id'].isnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


<b> Now let's look at the image prediction dataframe

In [33]:
image_prediction_df.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
791,690690673629138944,https://pbs.twimg.com/media/CZXTgKkWwAA5UZJ.jpg,1,bath_towel,0.194532,False,radiator,0.127776,False,Maltese_dog,0.089625,True
1354,760252756032651264,https://pbs.twimg.com/media/Coz12OLWgAADdys.jpg,1,radio_telescope,0.155279,False,dam,0.154515,False,crane,0.098040,False
1670,813081950185472002,https://pbs.twimg.com/media/C0ilsa1XUAEHK_k.jpg,2,Doberman,0.909951,True,kelpie,0.042649,True,miniature_pinscher,0.023004,True
351,672523490734551040,https://pbs.twimg.com/media/CVVIjGbWwAAxkN0.jpg,1,golden_retriever,0.565981,True,chow,0.081212,True,Irish_terrier,0.061596,True
970,706538006853918722,https://pbs.twimg.com/media/Cc4gjxqW4AIoThO.jpg,1,chow,0.541794,True,Pembroke,0.094918,True,Pomeranian,0.085439,True
1123,727175381690781696,https://pbs.twimg.com/media/ChdyJvdWwAA5HGd.jpg,2,flat-coated_retriever,0.656463,True,Great_Dane,0.084766,True,Labrador_retriever,0.058909,True
1908,852672615818899456,https://pbs.twimg.com/media/C9VNNp1XkAEWRFb.jpg,1,golden_retriever,0.711235,True,otterhound,0.068235,True,Sussex_spaniel,0.046562,True
828,693590843962331137,https://pbs.twimg.com/media/CaAhMb1XEAAB6Bz.jpg,1,dining_table,0.383448,False,grey_fox,0.103191,False,Siamese_cat,0.098256,False
978,707059547140169728,https://pbs.twimg.com/media/Cc_64zVWEAAeXs7.jpg,1,Samoyed,0.897312,True,Great_Pyrenees,0.039180,True,kuvasz,0.019516,True
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True


In [34]:
image_prediction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [35]:
# Checking the tweets having common image url
sum(image_prediction_df['jpg_url'].duplicated())

66

In [36]:
# Checking the accuracy of the predictions by the neural network
print(image_prediction_df['p1_dog'].value_counts())
print(image_prediction_df['p2_dog'].value_counts())
print(image_prediction_df['p3_dog'].value_counts())

True     1532
False     543
Name: p1_dog, dtype: int64
True     1553
False     522
Name: p2_dog, dtype: int64
True     1499
False     576
Name: p3_dog, dtype: int64


<b> Now let's look at the tweet_json dataframe

In [37]:
tweet_json.sample(10)

,tweet_id,favorite_count,retweet_count,followers_count,friends_count,source,retweeted_status,url
172,857062103051644929,0,170,8188704,12,Twitter for iPhone,This is a retweet,This is a retweet
107,870804317367881728,32743,5999,8188698,12,Twitter for iPhone,Original tweet,https://t.co/oZ7H2eO79W
579,798686750113755136,0,2482,8188711,12,Twitter for iPhone,This is a retweet,This is a retweet
487,813051746834595840,21995,7831,8188711,12,Twitter for iPhone,Original tweet,https://t.co/AIMF8ouzvl
185,855245323840757760,0,6010,8188704,12,Twitter for iPhone,This is a retweet,This is a retweet
2308,666353288456101888,211,71,8188744,12,Twitter for iPhone,Original tweet,https://t.co/tltQ5w9aUO
777,773191612633579521,10436,4370,8188713,12,Twitter for iPhone,Original tweet,https://t.co/9HEVPJEioD
1596,684969860808454144,2439,388,8188725,12,Twitter for iPhone,Original tweet,u
496,811627233043480576,13426,3315,8188711,12,Twitter for iPhone,Original tweet,https://t.co/IU8S0n4oxn
2120,669970042633789440,301,60,8188738,12,Twitter for iPhone,Original tweet,https://t.co/aeURGO9Qs8


In [38]:
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2333 entries, 0 to 2332
Data columns (total 8 columns):
tweet_id            2333 non-null object
favorite_count      2333 non-null int64
retweet_count       2333 non-null int64
followers_count     2333 non-null int64
friends_count       2333 non-null int64
source              2333 non-null object
retweeted_status    2333 non-null object
url                 2333 non-null object
dtypes: int64(4), object(4)
memory usage: 145.9+ KB


In [39]:
# Checking the retweeted status count
tweet_json['retweeted_status'].value_counts()

Original tweet       2168
This is a retweet     165
Name: retweeted_status, dtype: int64

In [40]:
# Checking the source count
tweet_json['source'].value_counts()

Twitter for iPhone     2199
Vine - Make a Scene      91
Twitter Web Client       33
TweetDeck                10
Name: source, dtype: int64

### Quality

<b>Now, we are gonna list down issues with content:</b>
<br><br>Validity,Accuracy,Completeness,Consistency
    
<br> <br>
<b> Twitter_archive_df dataframe</b><br><br>
1.Keep the original ratings that have images. Churn out the retweets.<br>
2.Erroneous datatypes(in_reply_to_status_id,in_reply_to_user_id,timestamp columns,etc)<br>
3.The numerator and denominator columns have invalid values<br>
4.Name column has invalid names i.e. a,an,'None' and less than 3 characters.<br>
5.In several columns, null objects are non-null(None to NaN).<br>
6.Unnecessary html tags in source column in place of utility name.<br><br>

<b> Image_prediction_df dataframe</b><br><br>
1.Some tweet_ids have the same jpg_url<br>
2.Many tweet_id(s) of twitter_prediction_df table are missing in image_prediction_df table(2075 rows instead of 2356).<br>

<b> Tweet_json dataframe</b><br><br>
1.Keep original tweets only. Remove duplicate tweets.<br>

### Tidiness


<b>Untidy data - Structural Issues

- Merge doggo,floofer,pupper,puppo columns in twitter_archive_df table called 'stage'
- Twitter_archive_df without any duplicates (i.e. retweets) will have empty retweeted_status_id, retweeted_status_user_id and retweeted_status_timestamp columns, which can be dropped.
- Join 'tweet_json' and 'image_prediction_df' to 'twitter_archive_df'

## Cleaning Data

<b> Cleaning our data is the third step in the data wrangling process. This is where we will fix the quality and tidiness issues we identified in the assessing step. 

In [41]:
# Copying the dataframes for cleaning process

twitter_archive_df_clean = twitter_archive_df.copy()
image_prediction_df_clean = image_prediction_df.copy()
tweet_json_clean = tweet_json.copy()

### Define

Keep the original ratings that have images. Churn out the retweets.


### Code

In [42]:
twitter_archive_df_clean = twitter_archive_df_clean[pd.isnull(twitter_archive_df_clean['retweeted_status_user_id'])]

### Test

In [43]:
print(sum(twitter_archive_df_clean.retweeted_status_user_id.value_counts()))

0


### Define

many tweet_id(s) of twitter_archive_df_clean table are missing in image_prediction_df_clean (image predictions) table

### Code

In [44]:
twitter_archive_df_clean = twitter_archive_df_clean[twitter_archive_df_clean.tweet_id.isin(image_prediction_df_clean.tweet_id)]

### Test

In [45]:
len(twitter_archive_df_clean[~twitter_archive_df_clean.tweet_id.isin(image_prediction_df_clean.tweet_id)])

0

### Define

Drop retweeted_status_id, retweeted_status_user_id and retweeted_status_timestamp columns from twitter_archive_df_clean table.

### Code

In [46]:
twitter_archive_df_clean.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1, inplace=True)

### Test

In [47]:
twitter_archive_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                 1994 non-null int64
in_reply_to_status_id    23 non-null float64
in_reply_to_user_id      23 non-null float64
timestamp                1994 non-null object
source                   1994 non-null object
text                     1994 non-null object
expanded_urls            1994 non-null object
rating_numerator         1994 non-null int64
rating_denominator       1994 non-null int64
name                     1994 non-null object
doggo                    1994 non-null object
floofer                  1994 non-null object
pupper                   1994 non-null object
puppo                    1994 non-null object
dtypes: float64(2), int64(3), object(9)
memory usage: 233.7+ KB


### Define

Convert in_reply_to_status_id and in_reply_to_user_id to data type integer. Convert timestamp to datetime data type.

### Code

In [48]:
twitter_archive_df_clean.in_reply_to_status_id = twitter_archive_df_clean.in_reply_to_status_id.fillna(0)
twitter_archive_df_clean.in_reply_to_user_id = twitter_archive_df_clean.in_reply_to_user_id.fillna(0)

twitter_archive_df_clean.in_reply_to_status_id = twitter_archive_df_clean.in_reply_to_status_id.astype(np.int64)
twitter_archive_df_clean.in_reply_to_user_id = twitter_archive_df_clean.in_reply_to_user_id.astype(np.int64)

twitter_archive_df_clean.timestamp = pd.to_datetime(twitter_archive_df_clean.timestamp)

### Test

In [49]:
twitter_archive_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                 1994 non-null int64
in_reply_to_status_id    1994 non-null int64
in_reply_to_user_id      1994 non-null int64
timestamp                1994 non-null datetime64[ns, UTC]
source                   1994 non-null object
text                     1994 non-null object
expanded_urls            1994 non-null object
rating_numerator         1994 non-null int64
rating_denominator       1994 non-null int64
name                     1994 non-null object
doggo                    1994 non-null object
floofer                  1994 non-null object
pupper                   1994 non-null object
puppo                    1994 non-null object
dtypes: datetime64[ns, UTC](1), int64(5), object(8)
memory usage: 233.7+ KB


### Define

Correct numerators.

### Code

In [50]:
twitter_archive_df_clean[['rating_numerator', 'rating_denominator']] = twitter_archive_df_clean[['rating_numerator','rating_denominator']].astype(float)

twitter_archive_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                 1994 non-null int64
in_reply_to_status_id    1994 non-null int64
in_reply_to_user_id      1994 non-null int64
timestamp                1994 non-null datetime64[ns, UTC]
source                   1994 non-null object
text                     1994 non-null object
expanded_urls            1994 non-null object
rating_numerator         1994 non-null float64
rating_denominator       1994 non-null float64
name                     1994 non-null object
doggo                    1994 non-null object
floofer                  1994 non-null object
pupper                   1994 non-null object
puppo                    1994 non-null object
dtypes: datetime64[ns, UTC](1), float64(2), int64(3), object(8)
memory usage: 233.7+ KB


In [51]:
twitter_archive_df_clean[['rating_numerator', 'rating_denominator']] = twitter_archive_df_clean[['rating_numerator','rating_denominator']].astype(float)

twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 883482846933004288), 'rating_numerator'] = 13.5
twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 786709082849828864), 'rating_numerator'] = 9.75
twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 778027034220126208), 'rating_numerator'] = 11.27
twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 681340665377193984), 'rating_numerator'] = 9.5
twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 680494726643068929), 'rating_numerator'] = 11.26

with pd.option_context('max_colwidth', 200):
    display(twitter_archive_df_clean[twitter_archive_df_clean['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

C:\Users\Satyaraj\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()


,tweet_id,text,rating_numerator,rating_denominator
45,883482846933004288,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",13.50,10.0
695,786709082849828864,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",9.75,10.0
763,778027034220126208,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,11.27,10.0
1712,680494726643068929,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,11.26,10.0


### Test

In [52]:
twitter_archive_df_clean.rating_numerator.value_counts()

12.00      450
10.00      419
11.00      396
13.00      261
9.00       151
8.00        95
7.00        52
14.00       35
5.00        32
6.00        32
3.00        19
4.00        16
2.00         9
1.00         5
0.00         2
24.00        1
9.75         1
84.00        1
13.50        1
143.00       1
420.00       1
50.00        1
80.00        1
45.00        1
60.00        1
44.00        1
144.00       1
88.00        1
11.26        1
165.00       1
11.27        1
1776.00      1
99.00        1
121.00       1
204.00       1
Name: rating_numerator, dtype: int64

### Define

 Correct denominators

<b>Manually correcting them first

Five tweets with denominator not equal to 10 for special circunstances. Update both numerators and denominators when necessary. Delete other five tweets because they do not have actual ratings.

### Code

In [53]:
twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 740373189193256964), 'rating_numerator'] = 14
twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 740373189193256964), 'rating_denominator'] = 10

twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 682962037429899265), 'rating_numerator'] = 10
twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 682962037429899265), 'rating_denominator'] = 10

twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 666287406224695296), 'rating_numerator'] = 9
twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 666287406224695296), 'rating_denominator'] = 10

twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 722974582966214656), 'rating_numerator'] = 13
twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 722974582966214656), 'rating_denominator'] = 10

twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 716439118184652801), 'rating_numerator'] = 13.5
twitter_archive_df_clean.loc[(twitter_archive_df_clean.tweet_id == 716439118184652801), 'rating_denominator'] = 10


twitter_archive_df_clean = twitter_archive_df_clean[twitter_archive_df_clean['tweet_id'] != 832088576586297345]
twitter_archive_df_clean = twitter_archive_df_clean[twitter_archive_df_clean['tweet_id'] != 810984652412424192]
twitter_archive_df_clean = twitter_archive_df_clean[twitter_archive_df_clean['tweet_id'] != 682808988178739200]
twitter_archive_df_clean = twitter_archive_df_clean[twitter_archive_df_clean['tweet_id'] != 835246439529840640]
twitter_archive_df_clean = twitter_archive_df_clean[twitter_archive_df_clean['tweet_id'] != 686035780142297088]


with pd.option_context('max_colwidth', 200):
    display(twitter_archive_df_clean[twitter_archive_df_clean['rating_denominator'] != 10][['tweet_id',
                                                                                      'text',
                                                                                      'rating_numerator',
                                                                                      'rating_denominator']])

,tweet_id,text,rating_numerator,rating_denominator
433,820690176645140481,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84.0,70.0
902,758467244762497024,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165.0,150.0
1120,731156023742988288,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204.0,170.0
1228,713900603437621249,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,99.0,90.0
1254,710658690886586372,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80.0,80.0
1274,709198395643068416,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",45.0,50.0
1351,704054845121142784,Here is a whole flock of puppers. 60/50 I'll take the lot https://t.co/9dpcw6MdWa,60.0,50.0
1433,697463031882764288,Happy Wednesday here's a bucket of pups. 44/40 would pet all at once https://t.co/HppvrYuamZ,44.0,40.0
1634,684225744407494656,"Two sneaky puppers were not initially seen, moving the rating to 143/130. Please forgive us. Thank you https://t.co/kRK51Y5ac3",143.0,130.0
1635,684222868335505415,Someone help the girl is being mugged. Several are distracting her while two steal her shoes. Clever puppers 121/110 https://t.co/1zfnTJLt55,121.0,110.0


<b> Programmatically correcting them

In [54]:
twitter_archive_df_clean['rating'] = 10 * twitter_archive_df_clean['rating_numerator'] / twitter_archive_df_clean['rating_denominator'].astype(float)

### Test

In [55]:
twitter_archive_df_clean.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rating
725,782722598790725632,0,0,2016-10-02 23:23:04+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Penny. She fought a bee and the bee wo...,https://twitter.com/dog_rates/status/782722598...,10.0,10.0,Penny,None,None,None,None,10.0
1030,745433870967832576,0,0,2016-06-22 01:50:58+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Lenox. She's in a wheelbarrow. Silly d...,https://twitter.com/dog_rates/status/745433870...,10.0,10.0,Lenox,doggo,None,None,None,10.0
2200,668655139528511488,0,0,2015-11-23 04:59:42+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Winifred. He is a Papyrus Hydrang...,https://twitter.com/dog_rates/status/668655139...,11.0,10.0,Winifred,None,None,None,None,11.0
1882,674999807681908736,674793399141146624,4196983835,2015-12-10 17:11:09+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Ok last one of these. I may try to make some m...,https://twitter.com/dog_rates/status/674999807...,13.0,10.0,None,None,None,None,None,13.0
1608,685641971164143616,0,0,2016-01-09 01:59:19+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Otis. He just passed a cop while going...,https://twitter.com/dog_rates/status/685641971...,7.0,10.0,Otis,None,None,pupper,None,7.0


### Define

Name column has invalid names i.e. a,an,'None' and less than 3 characters.

### Code


In [56]:
twitter_archive_df_clean['name'][twitter_archive_df_clean['name'].str.match('[a-z]+')] = 'None'

C:\Users\Satyaraj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Test

In [57]:
twitter_archive_df_clean.name[twitter_archive_df_clean.name == 'None'].value_counts()

None    644
Name: name, dtype: int64

In [58]:
# Sort ascending by name to check if there are more names starting with a lowercase alphabet
twitter_archive_df_clean.name.value_counts().sort_index(ascending=False)

Zuzu            1
Zooey           1
Zoey            3
Zoe             1
Ziva            1
Zeus            1
Zeke            3
Zeek            1
Zara            1
Yogi            2
Yoda            1
Wyatt           3
Wishes          1
Winston         8
Winnie          4
Winifred        1
Wilson          3
Willy           1
Willow          1
Willie          1
William         1
Willem          1
Wiggles         1
Wesley          1
Watson          2
Walter          3
Wally           2
Wallace         3
Walker          1
Waffles         3
               ..
Apollo          1
Antony          1
Anthony         1
Anna            1
Angel           1
Andy            1
Andru           1
Anakin          1
Amélie          1
Amy             1
Ambrose         1
Amber           1
Alice           2
Alfy            1
Alfie           4
Alf             1
Alexanderson    1
Alexander       1
Alejandro       1
Aldrick         1
Albus           2
Albert          2
Al              1
Akumi           1
Aja       

### Define

Strip all html anchor tags (i.e. <a..></a>) in source column and retain just the text in between the tags. Convert the datatype from string to categorical.

### Code

In [59]:
twitter_archive_df_clean.source = twitter_archive_df_clean.source.str.replace(r'<(?:a\b[^>]*>|/a>)', '')

### Test

In [60]:
twitter_archive_df_clean.source.value_counts()

Twitter for iPhone    1954
Twitter Web Client      28
TweetDeck               11
Name: source, dtype: int64

### Define

Merge the doggo, floofer, pupper and puppo columns to a stage column. Convert the datatype from string to categorical as it helps with analysis and visualization and saves memory on disk.

Drop the doggo, floofer, pupper and puppo columns.

### Code

In [61]:
# merge the doggo, floofer, pupper and puppo columns to a stage column
twitter_archive_df_clean['stage'] = twitter_archive_df_clean[['doggo', 'floofer', 'pupper', 'puppo']].max(axis=1)

In [62]:
# convert the datatype from string to categorical
twitter_archive_df_clean.stage = twitter_archive_df_clean.stage.astype('category')

In [63]:
# drop the doggo, floofer, pupper and puppo columns
twitter_archive_df_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)

### Test

In [64]:
twitter_archive_df_clean.stage.value_counts()

None       1687
pupper      212
doggo        63
puppo        23
floofer       8
Name: stage, dtype: int64

### Define

Image_prediction_df_clean - Drop 66 jpg_url duplicated

### Code

In [65]:
image_prediction_df_clean = image_prediction_df_clean.drop_duplicates(subset=['jpg_url'], keep='last')

### Test

In [66]:
sum(image_prediction_df_clean['jpg_url'].duplicated())

0

### Define

Image_prediction - Create 1 column for image prediction and 1 column for confidence level

### Code

In [67]:
dog_type = []
confidence_list = []

def image(image_prediction_df_clean):
    if image_prediction_df_clean['p1_dog'] == True:
        dog_type.append(image_prediction_df_clean['p1'])
        confidence_list.append(image_prediction_df_clean['p1_conf'])
    elif image_prediction_df_clean['p2_dog'] == True:
        dog_type.append(image_prediction_df_clean['p2'])
        confidence_list.append(image_prediction_df_clean['p2_conf'])
    elif image_prediction_df_clean['p3_dog'] == True:
        dog_type.append(image_prediction_df_clean['p3'])
        confidence_list.append(image_prediction_df_clean['p3_conf'])
    else:
        dog_type.append('Error')
        confidence_list.append('Error')

#series objects having index the image_prediction_clean column.        
image_prediction_df_clean.apply(image, axis=1)


image_prediction_df_clean['dog_type'] = dog_type
image_prediction_df_clean['confidence_list'] = confidence_list

In [68]:
#drop rows that has prediction_list 'error'
image_prediction_df_clean = image_prediction_df_clean[image_prediction_df_clean['dog_type'] != 'Error']


### Test

In [69]:
image_prediction_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1691 entries, 0 to 2073
Data columns (total 14 columns):
tweet_id           1691 non-null int64
jpg_url            1691 non-null object
img_num            1691 non-null int64
p1                 1691 non-null object
p1_conf            1691 non-null float64
p1_dog             1691 non-null bool
p2                 1691 non-null object
p2_conf            1691 non-null float64
p2_dog             1691 non-null bool
p3                 1691 non-null object
p3_conf            1691 non-null float64
p3_dog             1691 non-null bool
dog_type           1691 non-null object
confidence_list    1691 non-null object
dtypes: bool(3), float64(3), int64(2), object(6)
memory usage: 163.5+ KB


### Define

 Image_prediction - Delete columns that won't be used for analysis

### Code

In [70]:
image_prediction_df_clean = image_prediction_df_clean.drop(['img_num', 'p1', 
                                                      'p1_conf', 'p1_dog', 
                                                      'p2', 'p2_conf', 
                                                      'p2_dog', 'p3', 
                                                      'p3_conf', 
                                                      'p3_dog'], 1)

### Test

In [71]:
print(list(image_prediction_df_clean))

['tweet_id', 'jpg_url', 'dog_type', 'confidence_list']


### Define

Tweet_json - keep 2174 original tweets

### Code

In [72]:
tweet_json_clean = tweet_json_clean[tweet_json_clean['retweeted_status'] == 'Original tweet']

### Test

In [73]:
tweet_json_clean['retweeted_status'].value_counts()

Original tweet    2168
Name: retweeted_status, dtype: int64

### Define

Tidiness - Change tweet_id to type int64 in order to merge with the other 2 tables

### Code

In [74]:
tweet_json_clean['tweet_id'] = tweet_json_clean['tweet_id'].astype('float')

### Test

In [75]:
tweet_json_clean['tweet_id'].dtypes

dtype('float64')

### Define

Get Dogs gender column from text column

### Code

In [76]:
male = ['He', 'he', 'him', 'his', "he's", 'himself']
female = ['She', 'she', 'her', 'hers', 'herself', "she's"]

dog_gender = []

for text in twitter_archive_df_clean['text']:
    # Male
    if any(map(lambda v:v in male, text.split())):
        dog_gender.append('male')
    # Female
    elif any(map(lambda v:v in female, text.split())):
        dog_gender.append('female')
    # If group or not specified
    else:
        dog_gender.append('NaN')


twitter_archive_df_clean['dog_gender'] = dog_gender

### Test

In [77]:
twitter_archive_df_clean.dog_gender.value_counts()

NaN       1132
male       636
female     225
Name: dog_gender, dtype: int64

### Define

Convert the null values to None type

### Code

In [78]:
twitter_archive_df_clean.loc[twitter_archive_df_clean['dog_gender'] == 'NaN', 'dog_gender'] = None
twitter_archive_df_clean.loc[twitter_archive_df_clean['rating_numerator'] == 'NaN', 'rating_numerator'] = 0

C:\Users\Satyaraj\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


### Test

In [79]:
twitter_archive_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1993 entries, 0 to 2355
Data columns (total 13 columns):
tweet_id                 1993 non-null int64
in_reply_to_status_id    1993 non-null int64
in_reply_to_user_id      1993 non-null int64
timestamp                1993 non-null datetime64[ns, UTC]
source                   1993 non-null object
text                     1993 non-null object
expanded_urls            1993 non-null object
rating_numerator         1993 non-null float64
rating_denominator       1993 non-null float64
name                     1993 non-null object
rating                   1993 non-null float64
stage                    1993 non-null category
dog_gender               861 non-null object
dtypes: category(1), datetime64[ns, UTC](1), float64(3), int64(3), object(5)
memory usage: 204.6+ KB


### Define

Tidiness - All tables should be part of one dataset

### Code

In [80]:
# Creating a new dataframe that merges twitter_archive_df_clean & image_prediction_df_clean first

twitter_df = pd.merge(twitter_archive_df_clean,image_prediction_df_clean,how='left',on=['tweet_id'])

#Accounting for  rows that only have pictures

twitter_df = twitter_df[twitter_df['jpg_url'].notnull()]

### Test

In [81]:
twitter_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1625 entries, 1 to 1992
Data columns (total 16 columns):
tweet_id                 1625 non-null int64
in_reply_to_status_id    1625 non-null int64
in_reply_to_user_id      1625 non-null int64
timestamp                1625 non-null datetime64[ns, UTC]
source                   1625 non-null object
text                     1625 non-null object
expanded_urls            1625 non-null object
rating_numerator         1625 non-null float64
rating_denominator       1625 non-null float64
name                     1625 non-null object
rating                   1625 non-null float64
stage                    1625 non-null category
dog_gender               727 non-null object
jpg_url                  1625 non-null object
dog_type                 1625 non-null object
confidence_list          1625 non-null object
dtypes: category(1), datetime64[ns, UTC](1), float64(3), int64(3), object(8)
memory usage: 204.9+ KB


In [82]:
twitter_df.drop(['dog_gender'],axis=1,inplace=True)

In [83]:
twitter_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1625 entries, 1 to 1992
Data columns (total 15 columns):
tweet_id                 1625 non-null int64
in_reply_to_status_id    1625 non-null int64
in_reply_to_user_id      1625 non-null int64
timestamp                1625 non-null datetime64[ns, UTC]
source                   1625 non-null object
text                     1625 non-null object
expanded_urls            1625 non-null object
rating_numerator         1625 non-null float64
rating_denominator       1625 non-null float64
name                     1625 non-null object
rating                   1625 non-null float64
stage                    1625 non-null category
jpg_url                  1625 non-null object
dog_type                 1625 non-null object
confidence_list          1625 non-null object
dtypes: category(1), datetime64[ns, UTC](1), float64(3), int64(3), object(7)
memory usage: 192.2+ KB


In [84]:
# Creating a new dataframe that merges twitter_df & tweet_json_clean

twitter_df_final = pd.merge(twitter_df,tweet_json_clean,how='left',on=['tweet_id'])

In [85]:
twitter_df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1625 entries, 0 to 1624
Data columns (total 22 columns):
tweet_id                 1625 non-null int64
in_reply_to_status_id    1625 non-null int64
in_reply_to_user_id      1625 non-null int64
timestamp                1625 non-null datetime64[ns, UTC]
source_x                 1625 non-null object
text                     1625 non-null object
expanded_urls            1625 non-null object
rating_numerator         1625 non-null float64
rating_denominator       1625 non-null float64
name                     1625 non-null object
rating                   1625 non-null float64
stage                    1625 non-null category
jpg_url                  1625 non-null object
dog_type                 1625 non-null object
confidence_list          1625 non-null object
favorite_count           1053 non-null float64
retweet_count            1053 non-null float64
followers_count          1053 non-null float64
friends_count            1053 non-null float64


## Storing Data

In [86]:
#Storing the clean dataframe in a CSV file
twitter_df_final.to_csv('twitter_archive_master.csv',index=False,encoding='utf-8')

### Analysing & Visualising Data is done in  act_report.ipynb notebook